# Stage 1: Importing project dependencies

In [0]:
import math
import random
import numpy as np
import statistics
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook, tqdm
from collections import deque
import os.path


# Stage 2: Building the AI Trader network

In [0]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=1500)
    self.model_name = model_name
    self.inventory_gp = []
    self.gamma = 0.95
    self.epsilon = 1
    self.epsilon_final = 0.001
    self.epsilon_decay = 0.995
    self.LR = 0.001
    self.TAU = 1e-3       # for soft update of target parameters
    self.t_step = 0
    self.update_every = 4  #udate the network again after 4 timestep
    
    self.local_network = self.model_builder()
    self.target_network = self.model_builder()
    
  def model_builder(self):
    
      model = tf.keras.models.Sequential()
      
      model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
      
      model.add(tf.keras.layers.Dense(units=64, activation='relu'))
      
      model.add(tf.keras.layers.Dense(units=128, activation='relu'))

      model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
      
      model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=self.LR))
      
      return model
    
  def trade(self, state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.local_network.predict(state)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
  
    batch = random.sample(self.memory, 64)  
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.target_network.predict(next_state)[0])
          
      target = self.local_network.predict(state)
      target[0][action] = reward
        
      self.local_network.fit(state, target, epochs=1, verbose=0)

    self.soft_update()


  def soft_update(self):
    for target_params, local_params in zip(self.target_network.trainable_variables, self.local_network.trainable_variables):
      target_params.assign(target_params * (1 - self.TAU) + local_params * self.TAU)


# Stage 3: Dataset preprocessing

##helper function

In [0]:
def sigmoid(x):
  return (1 / (1 + np.exp(-x)))

##Price format function

In [0]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

##Loading ANN for trend analysis(classification) and preprocessing it

### Load the pretrained model

In [5]:
with open('GP classification with factor.json', 'r') as f:
    model_json = f.read()

model = tf.keras.models.model_from_json(model_json)
# load weights into new model
model.load_weights("GP classification with factor.json.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


### loading the dataset

In [0]:
dataset_train = pd.read_csv('GRAE Historical Data 2018 practice.csv')
sentiment = dataset_train.iloc[:, 16:17].values
X_classifier = dataset_train.iloc[:, [7,11,12,13,14]].values
y_classifier = dataset_train.iloc[:, 15:16].values

### feature scaling

In [0]:
sc_1 = MinMaxScaler() 
X_classifier = sc_1.fit_transform(X_classifier)

## Loading LSTM for trend analysis(regression) and preprocessing it

### Load the pretrain model

In [0]:
with open('gp prediction with factor.json', 'r') as f:
    modelgp_json = f.read()

regressor = tf.keras.models.model_from_json(modelgp_json)

# load weights into new model
regressor.load_weights("gp prediction with factor.json.h5")

### Loading the data

In [0]:
dataset_test_1 = pd.read_csv('GRAE Historical Data 2009-2017.csv').iloc[:,[1,7,11,12,13,14]]
dataset_test_1 = dataset_test_1.iloc[-60:,:] 
dataset_test_2 = pd.read_csv('GRAE Historical Data 2018 practice.csv').iloc[:,[1,7,11,12,13,14]]
dataset_test = pd.concat([dataset_test_1, dataset_test_2], axis = 0, ignore_index=True, sort=False)
test_set_1 = dataset_test.iloc[:,1:].values
test_set_y = dataset_test.iloc[:, 0:1].values

### Feature scaling

In [0]:
inputs = test_set_1[:,:]
sc_2 = MinMaxScaler(feature_range = (0, 1))
inputs = sc_1.transform(inputs)
test_set_scaled_y = sc_2.fit_transform(test_set_y)

### Making 60 timesteps and 5 factors data for LSTM

In [0]:
X_regressor = []
for i in range(60, len(test_set_1)):
    X_regressor.append(inputs[i-60:i, :])

X_regressor[0] = np.reshape(X_regressor[0], (1,-1))
array = np.reshape(X_regressor[0],(1,60,-1))
 
for i in range(1,len(X_regressor)):
    X_regressor[i] = np.reshape(X_regressor[i],(1,-1))
    X_regressor[i] = np.reshape(X_regressor[i],(1,60,-1))
    array = np.vstack((array,X_regressor[i]))

X_regressor = array

y_regressor = []
for i in range(60,len(test_set_scaled_y)):
    y_regressor.append(test_set_scaled_y[i,0])

y_regressor = np.array(y_regressor)
y_regressor = np.reshape(y_regressor, (-1,1))

##State creator

In [0]:
def state_creator(data, timestep, window_size,obj):
  
  starting_id = timestep - window_size + 1
  
  #catching the trend
  y_pred = model.predict(np.reshape(X_classifier[timestep],(1,-1)))
  
  if y_pred>0.5:
    y_pred = 1
  else:
    y_pred = 0

  #predicting the price
  predicted_next_price = sc_2.inverse_transform(regressor.predict(np.reshape(X_regressor[timestep],(1,X_regressor[timestep].shape[0],X_regressor[timestep].shape[1]))))[0,0]
  if timestep > 0:
    predicted_present_price = sc_2.inverse_transform(regressor.predict(np.reshape(X_regressor[timestep-1],(1,X_regressor[timestep-1].shape[0],X_regressor[timestep-1].shape[1]))))[0,0]
  else:
      predicted_present_price = predicted_next_price
      
  diffrence = predicted_next_price - predicted_present_price
  
  if diffrence>0:
    diffrence = 1
  else:
    diffrence = 0
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])

  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  score = sentiment[timestep,0]
  state.append(y_pred)
  state.append(diffrence)
  state.append(score)
  state = np.array([state])
  return state

##Loading a dataset

In [0]:
data = list(pd.read_csv( "GRAE Historical Data 2018 practice.csv")["Price"])

# Stage 4: Training the AI Trader

##Setting hyper parameters

In [0]:
window_size = 10
episodes = 1000
trend_regression = 1
trend_classification = 1
sentiment_score = 1
sharpe_ratio_list = []
portfolio_list = []

batch_size = 64
data_samples = len(data) - 1


##Defining the Trader model

In [0]:
trader = AI_Trader(window_size+trend_regression+trend_classification+sentiment_score)

##Training loop

In [0]:
episode = 0
while True:
  episode += 1  
  print("Episode: {}".format(episode))
  state = state_creator(data, 0, window_size + 1,trader)
  total_profit = 0
  trader.inventory_gp = []
  stock_return_list = []
  profit_list = []
      
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1,trader)
    reward = 0
    if action == 1: #Buying gp
        trader.inventory_gp.append(data[t])
        print("AI Trader bought: ", stocks_price_format(data[t]))

    if action == 2 and len(trader.inventory_gp) > 0: #Selling gp
      buy_price = trader.inventory_gp.pop(0)
      total_profit += (data[t] - buy_price)
      #reward = max((data[t] - buy_price),-1)
      profit_list.append(data[t] - buy_price)
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price))

    if len(portfolio_list) and portfolio_list[0]:
        portfolio = (sum(trader.inventory_gp)-portfolio_list[0])/portfolio_list[0]
        portfolio_list.append(portfolio)
        
    else:
        portfolio = sum(trader.inventory_gp)
        portfolio_list.append(portfolio)
        
    if t == data_samples - 1:
      done = True
      if len(profit_list)>0:
        reward = statistics.mean(profit_list)/statistics.stdev(profit_list)
      else:
        reward = 0
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      sharpe_ratio = statistics.mean(portfolio_list)/statistics.stdev(portfolio_list)
      sharpe_ratio_list.append(sharpe_ratio)
      print("SHARPE RATIO: {}".format(sharpe_ratio))
      print("REWARD IS: {}".format(reward))
      print("########################")

    
    f = open("file2.txt", "w")
    f.write(str(trader.epsilon))
    f.close()
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)

  trader.epsilon = max(trader.epsilon_final, trader.epsilon_decay*trader.epsilon)  #decay the value of epsilon 

  #saving the model parameter
  trader_local = trader.local_network.to_json()
  with open("AI_trader_local_network_gp.json", "w") as json_file:
      json_file.write(trader_local)

  trader_target = trader.target_network.to_json()
  with open("AI_trader_target_network_gp.json", "w") as json_file:
      json_file.write(trader_target)

  ### Saving network weights
  trader.local_network.save_weights("AI_trader_local_network_weights_gp.json.h5")
  trader.target_network.save_weights("AI_trader_target_network_weights_gp.json.h5")


  0%|          | 0/240 [00:00<?, ?it/s]

Episode: 1
AI Trader bought:  $ 472.600000


  8%|▊         | 19/240 [00:00<00:02, 86.22it/s]

AI Trader sold:  $ 470.900000  Profit: - $ 1.700000
AI Trader bought:  $ 476.000000
AI Trader bought:  $ 473.600000
AI Trader sold:  $ 479.800000  Profit: $ 3.800000
AI Trader sold:  $ 481.400000  Profit: $ 7.800000
AI Trader bought:  $ 483.300000
AI Trader sold:  $ 487.700000  Profit: $ 4.400000
AI Trader bought:  $ 494.400000
AI Trader sold:  $ 503.800000  Profit: $ 9.400000
AI Trader bought:  $ 505.900000
AI Trader sold:  $ 510.700000  Profit: $ 4.800000


 16%|█▋        | 39/240 [00:00<00:02, 92.11it/s]

...

 98%|█████████▊| 235/240 [01:29<00:01,  2.71it/s]

AI Trader sold:  $ 356.500000  Profit: - $ 10.900000


 99%|█████████▉| 237/240 [01:30<00:01,  2.66it/s]

AI Trader sold:  $ 359.600000  Profit: - $ 1.000000


 99%|█████████▉| 238/240 [01:30<00:00,  2.64it/s]

##sharpe ratio plot

In [0]:
plt.plot(sharpe_ratio_list)
plt.title('sharpe_ratio (Training set)')
plt.xlabel('time')
plt.ylabel('sharpe ratio')
plt.show()

# Rough section

In [0]:







"""**loading the weight**"""

if os.path.exists("AI_trader_weights_v3.0_1100.json.h5")== True:
    trader.model.load_weights("AI_trader_weights_v3.0_1100.json.h5")

"""**loading the test set**"""

data_sqph_test = list(pd.read_csv( "SQPH Historical Data 2018-2019.csv")["Price"])
data_gp_test = list(pd.read_csv( "GRAE Historical Data 2018-2019.csv")["Price"])

"""**Loading the pretrained LSTM for prediction**"""

with open('SQPH prediction.json', 'r') as f:
    modelsqph_json = f.read()

model_sqph = tf.keras.models.model_from_json(modelsqph_json)

with open('GP prediction.json', 'r') as f:
    modelgp_json = f.read()

model_gp = tf.keras.models.model_from_json(modelgp_json)

# load weights into new model
model_sqph.load_weights("SQPH prediction.json.h5")
model_gp.load_weights("GP prediction.json.h5")

dataset_test_sqph = pd.read_csv('SQPH Historical Data 2018-2019.csv')
real_stock_price_sqph = dataset_test_sqph.iloc[:, 1:2].values
dataset_train_sqph = pd.read_csv('SQPH Historical Data2017-2018.csv')
dataset_total_sqph = pd.concat((dataset_train_sqph['Price'], dataset_test_sqph['Price']), axis = 0)

dataset_test_gp = pd.read_csv('GRAE Historical Data 2018-2019.csv')
real_stock_price_gp = dataset_test_gp.iloc[:, 1:2].values
dataset_train_gp = pd.read_csv('GRAE Historical Data 2017-2018.csv')
dataset_total_gp = pd.concat((dataset_train_gp['Price'], dataset_test_gp['Price']), axis = 0)

inputs_sqph = dataset_total_sqph[len(dataset_total_sqph) - len(dataset_test_sqph) - 60:].values
inputs_gp = dataset_total_gp[len(dataset_total_gp) - len(dataset_test_gp) - 60:].values
inputs_sqph = inputs_sqph.reshape(-1,1)
inputs_gp = inputs_gp.reshape(-1,1)
sc = MinMaxScaler(feature_range = (0, 1))
inputs_sqph = sc.fit_transform(inputs_sqph)
inputs_gp = sc.transform(inputs_gp)
X_test_sqph = []
for i in range(60, 60+dataset_test_sqph.shape[0]):
    X_test_sqph.append(inputs_sqph[i-60:i, 0])
X_test_sqph = np.array(X_test_sqph)
X_test_sqph = np.reshape(X_test_sqph, (X_test_sqph.shape[0], X_test_sqph.shape[1], 1))

X_test_gp = []
for i in range(60, 60+dataset_test_gp.shape[0]):
    X_test_gp.append(inputs_gp[i-60:i, 0])
X_test_gp = np.array(X_test_gp)
X_test_gp = np.reshape(X_test_gp, (X_test_gp.shape[0], X_test_gp.shape[1], 1))

"""**Testing the model**"""

for episode in range(1, 2):
  money = 100000
    
  state = state_creator(data_gp_test, data_sqph_test, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory_gp = []
  trader.inventory_sqph = []

  for t in tqdm(range(len(data_sqph_test)-1)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data_gp_test, data_sqph_test, t+1, window_size + 1)
    
    if action == 1: #Buying gp
        if money >= data_gp_test[t]:
            trader.inventory_gp.append(data_gp_test[t])
            money -= data_gp_test[t]
            print("AI Trader bought: ", stocks_price_format(data_gp_test[t]),"Total money: ",money)

    elif action == 2 and len(trader.inventory_gp) > 0: #Selling gp
      buy_price = trader.inventory_gp.pop(0)
      
      total_profit += data_gp_test[t] - buy_price
      money += total_profit
      print("AI Trader sold: ", stocks_price_format(data_gp_test[t]), " Profit: " + stocks_price_format(data_gp_test[t] - buy_price),"Total money: ",money)

    if action == 3: #Buying sqph
        if money >= data_sqph_test[t]:
            trader.inventory_sqph.append(data_sqph_test[t])
            money -= data_sqph_test[t]
            print("AI Trader bought: ", stocks_price_format(data_sqph_test[t]),"Total money: ",money)

    elif action == 4 and len(trader.inventory_sqph) > 0: #Selling sqph
      buy_price = trader.inventory_sqph.pop(0)
      
      total_profit += data_sqph_test[t] - buy_price
      money += total_profit
      print("AI Trader sold: ", stocks_price_format(data_sqph_test[t]), " Profit: " + stocks_price_format(data_sqph_test[t] - buy_price),"Total money: ",money)
    
    if t == len(data_sqph_test)-2 :
      done = True
    else:
      done = False
          
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("TOTAL money: {}".format(money))
      print("########################")

